# Instalação

In [ ]:
#!pip install xcompact3d_toolbox

In [ ]:
#!pip install hvplot

In [ ]:
#!pip install datashader

# Visualizando e Pós-processando uma Simulação

Esse notebook é um tutorial de como carregar e pós-processar os dados gerados pela simulação de um escoamento ao redor de um cilindro.

Como primeiro passo se importa as bibliotecas necessárias.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import xarray as xr
import hvplot.xarray
import xcompact3d_toolbox as x3d

import warnings
warnings.filterwarnings('ignore')

# Carregando o banco de dados

Na linha abaixo são lidos os dados da configuração da simulação.

In [ ]:
prm = x3d.Parameters(loadfile="input.i3d")

Visualizando os parâmetros da simulação.

In [ ]:
print(prm)

Neste notebook, devido ao tamnho dos dados gerados pela simulação, cerca de 7.7 GB, se trabalhará apenas com um plano no eixo xy.

Os dados utilizados nesse notebook foram previamente tratados e salvados no formato netCDF.

A linha de código abaixo carrega na memória o contéudo do arquivo cylinder_plane_xy.nc em um xarray.Dataset.

In [ ]:
ds = xr.open_dataset("cylinder_plane_xy.nc")

# Visualizando os dados

Na célula abaixo se visualiza o contéudo do xarray.Dataset.

In [ ]:
ds

Esse banco de dados é composto pelo campo de velocidade para as componentes x e y, ux e uy, o campo de pressão pp e epsilon, representando a geometria.

Pode-se visualizar a variável epsilon usando o comando plot.

In [ ]:
ax = ds.epsilon.plot(x="x", y="y")
ax.axes.set_title("")
ax.axes.set_aspect('equal');

Selecionando um ponto dentro do domínio pode-se acompanhar a variação do campo de velocidade ao longo do tempo.

Veja o código abaixo:

In [ ]:
fig, ax = plt.subplots(figsize = (6,4))

# Plot

ds.ux.sel(x=10.0, y=5.0, method="nearest").plot(x="t", label = 'ux', ax = ax)
ds.uy.sel(x=10.0, y=5.0, method="nearest").plot(x="t", label = 'uy', ax = ax)

# Legend

ax.legend(loc = 'lower left',frameon = False)

# Title

ax.set_title("")

# Axis labels

ax.set_ylabel("u");

Utilizando as funcionalidades da biblioteca hvPlot pode-se gerar um gráfico interativo para a variável ux.

In [ ]:
ds.ux.hvplot(x="t", widget_location="bottom")

Pode-se visualizar como as varáveis ux e uy se comportam ao longo do tempo com o comando plot. Usando o método isel pode-se montar o gráfico para o campo de velocidade a cada 20 intervalos de tempo.

In [ ]:
# Plot

a = ds.ux.isel(t=slice(None,None,20)).plot(x="x", y = "y", col = "t", cmap = 'turbo', rasterized = True)
b = ds.uy.isel(t=slice(None,None,20)).plot(x="x", y = "y", col = "t", cmap = 'turbo', rasterized = True)

# Aspect ratio

for ax in a.axes.flat:
    ax.axes.set_aspect("equal")
for ax in b.axes.flat:
    ax.axes.set_aspect("equal")

Pode-se montar uma animação para a variável ux com as funcionalidades da biblioteca hvPlot.

In [ ]:
ds.ux.sel(t=slice(None, None)).hvplot(
    x="x",
    y="y",
    aspect="equal",
    clim=(-1.07, 1.66),
    rasterize=True,
    cmap="turbo",
    widget_type="scrubber",
    widget_location="bottom",
    title="Escoamento ao redor de um cilindro",
)

# Pós-processamento

Nesta seção do notebook, se explorará as capacidados do toolbox no pós-processamento dos resultados. Como exemplo se calculou a vorticidade $\omega_z$ definida pela equação:

$$
\omega_z = \dfrac{\partial u_y}{\partial x}  - \dfrac{\partial u_x}{\partial y}.
$$

Antes de se aplicar a derivada parcial, defini-se a condição de contorno para para o campo de velocidade

In [ ]:
ds["ux"].attrs["BC"] = prm.get_boundary_condition("u")
ds["uy"].attrs["BC"] = prm.get_boundary_condition("u")

Na próxima célula se calcula as derivadas parciais utilizando um esquema de quarta ordem.

In [ ]:
ds["vort"] = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")

Pode-se adicionar os atributos da variável recem calculada da forma:

In [ ]:
ds["vort"].attrs = {"name": "wz", "long_name": "Vorticidade", "units": "-"}

Agora pode-se montar os gráficos da vorticidade no plano xy ao longo do tempo.

In [ ]:
g = ds.vort.isel(t=slice(None, None, 10)).plot(
    x="x", y="y", col="t", col_wrap=6, cmap="turbo", rasterized=True, robust=True
)
for ax in g.axes.flat:
    ax.axes.set_aspect("equal")

Por fim, pode-se fazer uma animação para visualizar a vorticidade.

In [ ]:
ds.vort.sel(t=slice(None, None)).hvplot(
    x="x",
    y="y",
    aspect="equal",
    clim=(-2.5, 2.5),
    rasterize=True,
    cmap="turbo",
    widget_type="scrubber",
    widget_location="bottom",
    title="Escoamento ao redor de um cilindro",
)